In [13]:
from functools import partial

import numpy as np
from scipy.interpolate import interpn

import chugunov_indicator_ as chug

In [14]:
data = chug.DetonationData("./data/subch_plt08582/")

yt : [INFO     ] 2024-09-22 21:56:57,592 Parameters: current_time              = 0.40003216130915
yt : [INFO     ] 2024-09-22 21:56:57,592 Parameters: domain_dimensions         = [ 640 1280    1]
yt : [INFO     ] 2024-09-22 21:56:57,596 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2024-09-22 21:56:57,596 Parameters: domain_right_edge         = [5.12000000e+09 1.02400000e+10 6.28318531e+00]


In [15]:
T, D = data.D_T_meshgrid(num=10)

In [16]:
T_, D_ = T[0], D[:, 0]

In [17]:
i = 1000
kwargs = {
    "abar": data.data["abar"][i],
    "zbar": data.data["zbar"][i],
    "z2bar": data.data["z2bar"][i],
    **data.data["ScreenFactors"][1]
}
kwargs

{'abar': 4.028776983649795,
 'zbar': 2.0143884920095214,
 'z2bar': 4.12949645671095,
 'z1': 2,
 'a1': 4,
 'z2': 6,
 'a2': 12}

In [18]:
minmax = {
    "abar": (data.data["abar"].min(), data.data["abar"].max()),
    "zbar": (data.data["zbar"].min(), data.data["zbar"].max()),
    "z2bar": (data.data["z2bar"].min(), data.data["z2bar"].max()),
    "z1": (1, 10),
    "z2": (1, 10)
}

minmax

{'abar': (4.028776676342192, 52.43965444432472),
 'zbar': (2.0143883383557393, 26.223830930940498),
 'z2bar': (4.1294958073363555, 713.3104742534674),
 'z1': (1, 10),
 'z2': (1, 10)}

In [19]:
linspaces_ = {k: np.linspace(*v, 10) for k, v in minmax.items()}

In [20]:
linspaces_["T"] = T_
linspaces_["D"] = D_

In [21]:
grids = np.meshgrid(*linspaces_.values())

In [22]:
parameter_space = {k: grids[j] for j, k in enumerate(linspaces_)}

In [23]:
parameter_space["abar"].shape

(10, 10, 10, 10, 10, 10, 10)

In [25]:
C = chug.parameters_from_vars(**parameter_space, percent=0.995)

In [26]:
display(C.shape, (C.min(), C.max()), C)

(10, 10, 10, 10, 10)

(19.785218833671035, 27.23579997524778)

array([[[[[20.57985336, 21.35766247, 21.90431083, ..., 22.69053053,
           22.69053053, 22.69053475],
          [21.35766247, 21.90454382, 22.69053008, ..., 23.23718311,
           23.23718733, 23.23741611],
          [21.90431083, 22.69053008, 22.69053475, ..., 23.78383525,
           23.7838357 , 24.02340237],
          ...,
          [22.69053053, 23.23718311, 23.78383525, ..., 24.5700554 ,
           24.5700554 , 24.57005962],
          [22.69053053, 23.23718733, 23.7838357 , ..., 24.5700554 ,
           24.57005962, 24.57028839],
          [22.69053475, 23.23741611, 24.02340237, ..., 24.57005962,
           24.57028839, 24.57847022]],

         [[21.91272564, 22.69076352, 23.23718311, ..., 24.02340282,
           24.02340704, 24.02363581],
          [22.69076352, 23.23741611, 23.7838357 , ..., 24.5700554 ,
           24.57028839, 24.57028839],
          [23.23718311, 23.7838357 , 24.02340704, ..., 25.11670753,
           25.11670798, 25.35627466],
          ...,
          [24.

In [46]:
np.save("C_array", C)
#np.load("C_array.npy")

In [66]:
i = 20000
test_kwargs = {
    "abar": data.data["abar"][i],
    "zbar": data.data["zbar"][i],
    "z2bar": data.data["z2bar"][i],
    **data.data["ScreenFactors"][2]
}

del test_kwargs["a1"]
del test_kwargs["a2"]

In [61]:
predict_C = partial(interpn, points=(*linspaces_.values(),)[:-2], values=C, method="cubic")

In [67]:
xi = np.array([*test_kwargs.values()])
xi

array([4.02877698, 2.01438849, 4.12949646, 2.        , 7.        ])

In [68]:
predict_C(xi=xi)

array([23.23718308])

In [70]:
chug.parameters_from_vars(*xi, *np.meshgrid(T_, D_), percent=0.995)

array(23.23718311)